<a href="https://colab.research.google.com/github/maahieummah/Jammu-Division-Flood-Damage-Dashboard/blob/main/Jammu_Division_Flood_Damage_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
# =====================================
# Jammu Division Flood Dashboard (with Map + CSV/PDF Download)
# =====================================

import pandas as pd
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import inch
from reportlab.lib.styles import getSampleStyleSheet
import tempfile

# -------------------------------
# 1️⃣ Data setup
# -------------------------------
data = {
    "District": [
        "Jammu", "Udhampur", "Kathua", "Reasi", "Doda", "Kishtwar",
        "Ramban", "Rajouri", "Poonch", "Samba"
    ],
    "Reported deaths / missing": [
        "Part of region total", "—", "Several deaths", "Few deaths", "Several deaths",
        "Many deaths (dozens+)", "Some deaths", "Some impact", "Some impact", "Some impact"
    ],
    "Houses — fully damaged": [414, 1895, 404, 249, 344, 190, 50, 30, 20, 10],
    "Houses — partially damaged": [1704, 3107, 1166, 636, 711, 283, 80, 40, 30, 20],
    "Other reported losses / notes": [
        "~414 ha crop loss; major infrastructure damage (bridges/roads).",
        "268 livestock deaths; 476 ha crops. (worst affected in division).",
        "Cloudbursts; ~1,200 houses affected; bridge/road loss.",
        "~408.9 ha crop loss reported.",
        "Multiple landslides (250+ across region).",
        "Cloudburst during pilgrimage; heavy livestock loss.",
        "Landslides & flash floods; few deaths.",
        "Minor damage compared to others.",
        "Damage to houses/roads; included in totals.",
        "Included in Jammu-division totals."
    ],
    "Sources": [
        "KashmirLife; New Indian Express",
        "KashmirLife",
        "KashmirLife; PTI / ThePrint",
        "KashmirLife; NDTV",
        "KashmirLife; Deccan Herald / PTI",
        "KashmirLife; DailyExcelsior; Reuters",
        "KashmirLife; NDTV",
        "KashmirLife",
        "KashmirLife; local reports",
        "KashmirLife; New Indian Express"
    ],
    # ✅ Approximate coordinates (latitude, longitude)
    "Latitude": [32.73, 32.92, 32.37, 33.08, 33.15, 33.32, 33.24, 33.38, 33.77, 32.57],
    "Longitude": [74.87, 75.14, 75.54, 74.83, 75.56, 75.78, 75.19, 74.31, 74.09, 75.12]
}
df = pd.DataFrame(data)

# -------------------------------
# 2️⃣ App setup
# -------------------------------
app = Dash(__name__, external_stylesheets=[dbc.themes.COSMO])
app.title = "Jammu Division Flood Dashboard"

# -------------------------------
# 3️⃣ Layout
# -------------------------------
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H2("🌊 Jammu Division Flood Damage Dashboard",
                        className="text-center text-primary mb-3"), width=12)
    ]),

    # Buttons
    dbc.Row([
        dbc.Col([
            dbc.ButtonGroup([
                dbc.Button("🔁 Show All", id="reset-btn", color="secondary"),
                dbc.Button("📊 Toggle Chart", id="toggle-chart", color="info"),
                dbc.Button("⬇️ Download CSV", id="btn-download-csv", color="success"),
                dbc.Button("📄 Download PDF", id="btn-download-pdf", color="danger"),
                dbc.Button("🗺️ Show Map", id="toggle-map", color="primary")
            ], className="mb-4")
        ], width=12, className="text-center")
    ]),

    dcc.Download(id="download-csv"),
    dcc.Download(id="download-pdf"),

    # Dropdown
    dbc.Row([
        dbc.Col([
            html.Label("Select District:", className="fw-bold"),
            dcc.Dropdown(
                id="district-dropdown",
                options=[{"label": d, "value": d} for d in df["District"]],
                value="Jammu",
                clearable=False,
                className="mb-4"
            )
        ], width=4, lg={"size": 3, "offset": 4}),
    ], justify="center"),

    # Summary Cards
    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H6("Fully Damaged Houses", className="text-danger text-center"),
                html.H3(id="fully-damaged", className="text-center fw-bold")
            ])
        ]), width=3),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H6("Partially Damaged Houses", className="text-warning text-center"),
                html.H3(id="partially-damaged", className="text-center fw-bold")
            ])
        ]), width=3),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H6("Deaths / Missing", className="text-dark text-center"),
                html.P(id="deaths-info", className="text-center fw-bold")
            ])
        ]), width=3),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H6("Sources", className="text-info text-center"),
                html.P(id="sources-info", className="small text-muted text-center")
            ])
        ]), width=3)
    ], justify="center", className="mb-4"),

    # Notes
    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H5("Detailed Notes / Loss Summary", className="text-primary"),
                html.P(id="notes-info", className="text-muted")
            ])
        ]), width=10)
    ], justify="center", className="mb-4"),

    # Dynamic Chart / Map
    dbc.Row([
        dbc.Col([
            html.H5("📊 Damage Visualization", className="text-primary text-center mb-2"),
            dcc.Graph(id="main-graph", style={"height": "500px"})
        ], width=10)
    ], justify="center")
], fluid=True, style={"padding": "20px"})


# -------------------------------
# 4️⃣ Callbacks
# -------------------------------
@app.callback(
    [Output("fully-damaged", "children"),
     Output("partially-damaged", "children"),
     Output("deaths-info", "children"),
     Output("notes-info", "children"),
     Output("sources-info", "children"),
     Output("main-graph", "figure")],
    [Input("district-dropdown", "value"),
     Input("toggle-chart", "n_clicks"),
     Input("toggle-map", "n_clicks")]
)
def update_dashboard(selected_district, toggle_chart, toggle_map):
    row = df[df["District"] == selected_district].iloc[0]
    chart_mode = (toggle_chart or 0) % 2
    map_mode = (toggle_map or 0) % 2

    # 🗺️ Show map view if map button clicked odd times
    if map_mode == 1:
        fig = px.scatter_mapbox(
            df,
            lat="Latitude",
            lon="Longitude",
            hover_name="District",
            hover_data={
                "Houses — fully damaged": True,
                "Houses — partially damaged": True
            },
            color="Houses — fully damaged",
            size="Houses — fully damaged",
            color_continuous_scale="Reds",
            zoom=7,
            height=500
        )
        fig.update_layout(
            mapbox_style="open-street-map",
            mapbox_center={"lat": 33.0, "lon": 75.2},
            margin={"r": 0, "t": 0, "l": 0, "b": 0}
        )
    else:
        # 📊 Show bar/pie chart toggle
        if chart_mode == 0:
            fig = px.bar(
                df,
                x="District",
                y=["Houses — fully damaged", "Houses — partially damaged"],
                barmode="group",
                labels={"value": "No. of Houses", "variable": "Damage Type"},
                title="Houses Damaged by District"
            )
        else:
            fig = px.pie(
                df,
                names="District",
                values="Houses — fully damaged",
                hole=0.3,
                title="Share of Fully Damaged Houses"
            )
        fig.update_layout(template="plotly_white", height=450)

    return (
        f"{row['Houses — fully damaged']:,}",
        f"{row['Houses — partially damaged']:,}",
        row["Reported deaths / missing"],
        row["Other reported losses / notes"],
        row["Sources"],
        fig
    )


# ✅ CSV download
@app.callback(
    Output("download-csv", "data"),
    Input("btn-download-csv", "n_clicks"),
    Input("district-dropdown", "value"),
    prevent_initial_call=True
)
def download_csv(n_clicks, district):
    return dcc.send_data_frame(df[df["District"] == district].to_csv,
                               f"{district}_Flood_Report.csv", index=False)


# ✅ PDF download
@app.callback(
    Output("download-pdf", "data"),
    Input("btn-download-pdf", "n_clicks"),
    Input("district-dropdown", "value"),
    prevent_initial_call=True
)
def download_pdf(n_clicks, district):
    row = df[df["District"] == district].iloc[0]
    styles = getSampleStyleSheet()
    story = [
        Paragraph("Jammu Division Flood Damage Report", styles["Title"]),
        Spacer(1, 0.2 * inch),
        Paragraph(f"<b>District:</b> {district}", styles["Heading2"]),
        Spacer(1, 0.1 * inch),
        Paragraph(f"<b>Reported deaths / missing:</b> {row['Reported deaths / missing']}", styles["Normal"]),
        Paragraph(f"<b>Houses — fully damaged:</b> {row['Houses — fully damaged']}", styles["Normal"]),
        Paragraph(f"<b>Houses — partially damaged:</b> {row['Houses — partially damaged']}", styles["Normal"]),
        Spacer(1, 0.2 * inch),
        Paragraph(f"<b>Other reported losses / notes:</b> {row['Other reported losses / notes']}", styles["Normal"]),
        Spacer(1, 0.2 * inch),
        Paragraph(f"<b>Sources:</b> {row['Sources']}", styles["Italic"])
    ]

    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
        doc = SimpleDocTemplate(tmp.name, pagesize=A4)
        doc.build(story)
        pdf_path = tmp.name

    return dcc.send_file(pdf_path)


# -------------------------------
# 5️⃣ Run app
# -------------------------------
if __name__ == "__main__":
    app.run(debug=True)

<IPython.core.display.Javascript object>